In [20]:
from collections import Counter

import pysam
import pybedtools
from Bio import SeqIO
import pandas as pd
import os

4 cases 

is read 1 and is not reverse:

    start of read one is start of cDNA

is read 1 and is reverse:

    end of read 1 is start of cDNA

is read 2 and is not reverse:

    start of read 2 is end of cDNA
    
is read 2 and is reverse:

    end of read2 is end of cDNA


In [21]:
def filer_short_reads(interval):
    return len(interval) == 21

def get_seq_logos(fn, out_file):
    hg19_chrom_sizes = "/projects/ps-yeolab/genomes/hg19/hg19.chrom.sizes"
    hg19_fasta = "/projects/ps-yeolab/genomes/hg19/chromosomes/all.fa"
    
    sam_file = pysam.Samfile(fn)
    
    cDNA_start_neg = []
    cDNA_start_pos = []
    cDNA_end_neg = []
    cDNA_end_pos = []

    for x, read in enumerate(sam_file):
        if read.is_read1 and not read.is_reverse:
            cDNA_start_neg.append(pybedtools.create_interval_from_list([sam_file.getrname(read.tid),
                                                                    read.positions[0],
                                                                    read.positions[0] + 1,
                                                                   0,
                                                                   0,
                                                                   "-"]))

        if read.is_read1 and read.is_reverse:
            cDNA_start_pos.append(pybedtools.create_interval_from_list([sam_file.getrname(read.tid),
                                                                    read.positions[-1],
                                                                    read.positions[-1] + 1,
                                                                   0,
                                                                   0,
                                                                   "+"]))

        if read.is_read2 and not read.is_reverse:
            cDNA_end_pos.append(pybedtools.create_interval_from_list([sam_file.getrname(read.tid),
                                                                  read.positions[0],
                                                                  read.positions[0] + 1,
                                                                 0,
                                                                   0,
                                                                   "+"]))

        if read.is_read2 and read.is_reverse:
            cDNA_end_neg.append(pybedtools.create_interval_from_list([sam_file.getrname(read.tid), 
                                                                  read.positions[-1], 
                                                                  read.positions[-1] + 1,
                                                                 0,
                                                                   0,
                                                                   "-"]))
    print "done getting reads"
    cDNA_start = pybedtools.BedTool(cDNA_start_pos + cDNA_start_neg)
    cDNA_end = cDNA_end = pybedtools.BedTool(cDNA_end_pos + cDNA_end_neg)
        
    #Expand out to 10bp around the cDNA start
    cDNA_start = cDNA_start.slop(b=10, g=hg19_chrom_sizes)
    #Remove reads that are near the start and end of a chromosome and will therefore mess with seqlogo
    cDNA_start = cDNA_start.filter(filer_short_reads).saveas()
    result = cDNA_start.sequence(fi=hg19_fasta, s=True, fo="cDNA_start.fasta")
    
    #Expand out to 10bp around the cDNA start
    cDNA_end = cDNA_end.slop(b=10, g=hg19_chrom_sizes)
    #Remove reads that are near the start and end of a chromosome and will therefore mess with seqlogo
    cDNA_end = cDNA_end.filter(filer_short_reads).saveas()
    result = cDNA_end.sequence(fi=hg19_fasta, s=True, fo="cDNA_end.fasta")

    out_cDNA_start = out_file + ".cDNA_start.pdf"
    out_cDNA_end = out_file + ".cDNA_end.pdf"
    
    !seqlogo -F PDF -Y -a -S -n -x "nt from start" -s -10 -c -k 1 -B 2 -T 1 -f cDNA_start.fasta > $out_cDNA_start
    !seqlogo -F PDF -Y -a -S -n -x "nt from end" -s -10 -c -k 1 -B 2 -T 1 -f cDNA_end.fasta > $out_cDNA_end

In [22]:
#PTBP Rep 2
file_dir = "/home/gpratt/projects/encode/analysis/ad-hoc/encode_bams"
get_seq_logos(os.path.join(file_dir, "ENCFF511GIO.bam"), "PTBP_K562_Rep1")
get_seq_logos(os.path.join(file_dir, "ENCFF227OUZ.bam"), "PTBP_K562_Rep2")            
get_seq_logos(os.path.join(file_dir, "ENCFF289RXU.bam"), "U2AF2_K562_Rep1")
get_seq_logos(os.path.join(file_dir, "ENCFF995HAG.bam"), "U2AF2_K562_Rep2")
get_seq_logos(os.path.join(file_dir, "ENCFF523UFV.bam"), "U2AF2_HepG2_Rep1")
get_seq_logos(os.path.join(file_dir, "ENCFF728QXQ.bam"), "U2AF2_HepG2_Rep2")
get_seq_logos(os.path.join(file_dir, "ENCFF282IRT.bam"), "TRA2A_K562_Rep1")
get_seq_logos(os.path.join(file_dir, "ENCFF111LPA.bam"), "TRA2A_K562_Rep2")

done getting reads
done getting reads
done getting reads
done getting reads
done getting reads


IOError: file `/home/gpratt/projects/encode/analysis/ad-hoc/encode_bams/ENCFF728QXQ.bam` not found

In [23]:
file_dir = "/projects/ps-yeolab2/encode/analysis/encode_v12"
get_seq_logos(os.path.join(file_dir, "254_02_BCCIP.merged.bam"), "254_02_BCCIP")

done getting reads


In [25]:
get_seq_logos(os.path.join(file_dir, "254_INPUT_GAGATTCC-TAATCTTA_L006_R1.unassigned.adapterTrim.round2.rmRep.rmDup.sorted.bam"), "254_INPUT_BCCIP")

done getting reads


In [ ]:
get_seq_logos(os.path.join(file_dir, "242_INPUT_CGCTCATT-ATAGAGGC_L005_R1.unassigned.adapterTrim.round2.rmRep.rmDup.sorted.bam"), "242_INPUT_U2AF2")
get_seq_logos(os.path.join(file_dir, "272_INPUT_CGCTCATT-GGCTCTGA_L002_R1.unassigned.adapterTrim.round2.rmRep.rmDup.sorted.bam"), "272_INPUT_U2AF2")